<a href="https://colab.research.google.com/github/ssingh-rookie/2015/blob/master/NLP_imdb_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
imdb_data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [16]:
imdb_data

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [0]:
train = imdb_data['train']
test = imdb_data['test']

In [0]:
train_sentences = []
train_labels = []

for s,l in train:
  train_sentences.append(str(s.numpy()))
  train_labels.append(str(l.numpy()))

In [31]:
#train_sentences[0]
train_labels[:10]

['0', '0', '0', '1', '1', '1', '0', '0', '0', '0']

In [0]:
test_sentences = []
test_labels = []

for s,l in test:
  test_sentences.append(str(s.numpy()))
  test_labels.append(str(l.numpy()))

###global variables for tokens

In [0]:
vocab_size = 10000
oov_token = "<OOV>"
embedding_dimension = 16
max_len = 120
trunc_type = "post"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [0]:
train_token = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
train_token.fit_on_texts(train_sentences)
word_index = train_token.word_index

In [50]:
train_sentences[:10]

['b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."',
 "b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot devel

In [0]:
sequences_tr = train_token.texts_to_sequences(train_sentences)
train_sequences = pad_sequences(sequences_tr, maxlen=max_len, truncating=trunc_type)

In [55]:
train_sequences.shape

(25000, 120)

In [0]:
sequences_test = train_token.texts_to_sequences(test_sentences)
test_sequences = pad_sequences(sequences_test, maxlen=max_len, truncating=trunc_type)


In [57]:
test_sequences.shape

(25000, 120)

In [0]:
np_train_labels = np.array(train_labels)
np_test_labels = np.array(test_labels)